# Проект: АиФ Доброе сердце (ETL/витрина данных)  

**Заказчик:** Фонд «АиФ. Доброе сердце»  
Фонд был создан в 2005-м году по инициативе еженедельной газеты «Аргументы и факты». За эти годы больше 10 тысяч подопечных со сложными диагнозами получили помощь с оплатой операций, лекарств и медоборудования. Фонд помогает тяжелобольным детям и взрослым из разных регионов России, а также развивает проекты по всесторонней поддержке подопечных семей — чтобы никто не оставался наедине с болезнью.  

**Цель проекта:** Фонд хочет лучше узнать своих благотворителей для более эффективной работы. Для этого основной задачей будет проведения RFM-анализа, когортного анализа (retention, LTV, средний чек), а также расчет основных маркетинговых и продуктовых мет ей.  

**Используемые библиотеки:**
 

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import text, create_engine 
import pandas as pd
import yadisk
from tqdm import tqdm

## Подключение к Яндекс диску и БД

In [ ]:
# загрузка переменной окружения с кодами доступа
load_dotenv(find_dotenv())               

In [ ]:
# подключение к Яндекс Диску
APP_ID = os.getenv('app_id')
SECRET_ID = os.getenv('secret_id')
TOKEN = os.getenv('ya_token')
y = yadisk.YaDisk(APP_ID, SECRET_ID, TOKEN)
y.check_token()

In [ ]:
# подключение к БД
db_config = {'user': os.getenv('USER'),                       
             'pwd': os.getenv('PASSWORD'),                           
             'host': os.getenv('HOST'),
             'port': os.getenv('PORT'),                                        
             'db': os.getenv('DATABASE')}    

engine = create_engine("postgresql+psycopg2://{user}:{pwd}@{host}:{port}/{db}".format(**db_config))
conn = engine.connect()

## Загрузка и предобработка данных

In [ ]:
# создание списка файлов формата .csv на загрузку
list_of_files = []
for el in list(y.listdir('AIF')):
  if el['path'].endswith('.csv'):
    list_of_files.append(el['path'])

In [ ]:
# определение пути загрузки
load_path = "C:/Users/aif/"
if not os.path.exists(load_path):
    os.mkdir(load_path)
os.chdir(load_path)

In [ ]:
# скачивание файла, если он не был загружен ранее
for file in tqdm(list_of_files):
    if file.split('/')[-1] not in os.listdir():
        y.download(file.split(':')[1], file.split('/')[-1])
    else:
        continue

In [ ]:
# загрузка файла со списком загруженных ранее датасетов
loaded = open('loaded.txt', 'a')

In [ ]:
# формирование датасетов из файлов при условии, что они не были скачены и загружены ранее

actions = []
orders = []
loaded_files = []

list_of_files = list(map(lambda x: x.split('/')[-1], list_of_files))

for file in tqdm(list_of_files):
    if file not in open('loaded.txt', 'r+').read():
        if 'actions' in file:
            df = pd.read_csv(file, sep=';')
            actions.append(df)
            loaded_files.append(file)
        else:
            df = pd.read_csv(file, sep=';')
            orders.append(df)
            loaded_files.append(file)

In [ ]:
# создание записи о том, что файл загружен в датасет

for file in loaded_files:
    loaded.write(f"{file}\n")

In [ ]:
# объединение датасетов
try:
    actions_df = pd.concat(actions, ignore_index=True)
    orders_df = pd.concat(orders, ignore_index=True)
except Exception as e:
    print(e)

In [ ]:
# удаление полных дубликатов
try:
    actions_df = actions_df.drop_duplicates(keep='last').reset_index(drop=True)
    orders_df = orders_df.drop_duplicates(keep='last').reset_index(drop=True)
except Exception as e:
    print(e)

In [ ]:
# удаление лишних столбцов
try: 
    actions_df = actions_df.drop(columns=['CustomerActionActionTemplateIdsSystemName',
                  'CustomerActionBrandIdsSystemName', 
                  'CustomerActionChannelIdsSystemName',
                  'CustomerActionCustomerIdsBackendID', 
                  'CustomerActionCustomerIdsWebsiteID'])
    orders_df = orders_df.drop(columns=['OrderAreaIdsExternalId',
                                       'OrderTransactionIdsExternalId',
                                       'OrderIdsBackendID',
                                       'OrderCustomFieldsNextPayDate',
                                       'OrderLineGiftCardAmount',
                                       'OrderLineGiftCardStatusIdsSystemName',
                                       'OrderLineId',
                                       'OrderLineLineId',
                                       'OrderCustomerIdsBackendID',
                                       'OrderCustomerIdsWebsiteID'])
except Exception as e:
    print(e)

In [ ]:
# приведение типов данных
try:
    actions_df['CustomerActionDateTimeUtc'] = actions_df['CustomerActionDateTimeUtc'].apply(lambda x: x.split(' ')[0])
    actions_df['CustomerActionDateTimeUtc'] = pd.to_datetime(actions_df['CustomerActionDateTimeUtc'], format='%d.%m.%Y')
    actions_df['CustomerActionCreationDateTimeUtc'] = actions_df['CustomerActionCreationDateTimeUtc'].apply(lambda x: x.split(' ')[0])
    actions_df['CustomerActionCreationDateTimeUtc'] = pd.to_datetime(actions_df['CustomerActionCreationDateTimeUtc'], format='%d.%m.%Y')

    orders_df['OrderFirstActionDateTimeUtc'] = orders_df['OrderFirstActionDateTimeUtc'].apply(lambda x: x.split(' ')[0])
    orders_df['OrderFirstActionDateTimeUtc'] = pd.to_datetime(orders_df['OrderFirstActionDateTimeUtc'], format='%d.%m.%Y')
except Exception as e:
    print(e)

## Загрузка данных в БД

In [ ]:
# загрузка датасета actions
try:
    actions_df.to_sql('actions', con=conn, index=False, if_exists='append')
except Exception as e:
    print(e)

In [ ]:
# загрузка датасета orders
try:
    orders_df.to_sql('orders', con=conn, index=False, if_exists='append')
except Exception as e:
    print(e)

In [ ]:
# функция для чтения запросов к БД
def sql_query(query):
    return pd.io.sql.read_sql(sql=text(query), con = conn)

### Выделение RFM-сегментов

RFM-сегментация проводится только среди действующих жертвователей, то есть по таблице orders_df

In [ ]:
rfm = """WITH base AS (SELECT "OrderCustomerIdsMindboxId", 
                     "OrderFirstActionDateTimeUtc", 
                     "OrderLinePriceOfLine"
              FROM orders
              WHERE "OrderLineStatusIdsExternalId" = 'Paid'),
fm_base AS (SELECT "OrderCustomerIdsMindboxId" AS usr,
              SUM("OrderLinePriceOfLine") AS monetary,
              COUNT("OrderLinePriceOfLine") AS frequency
              FROM base
              GROUP BY "OrderCustomerIdsMindboxId"),
f AS (SELECT PERCENTILE_DISC(0.30) WITHIN GROUP (ORDER BY frequency) AS fr_fs,
             PERCENTILE_DISC(0.70) WITHIN GROUP (ORDER BY frequency) AS fr_sn
       FROM fm_base),
m AS (SELECT PERCENTILE_DISC(0.30) WITHIN GROUP (ORDER BY monetary) AS mn_fs,
             PERCENTILE_DISC(0.70) WITHIN GROUP (ORDER BY monetary) AS mn_sn
       FROM fm_base),
rec AS (SELECT DISTINCT "OrderCustomerIdsMindboxId" AS usr,
             LAST_VALUE("OrderFirstActionDateTimeUtc") OVER(PARTITION BY "OrderCustomerIdsMindboxId" ORDER BY "OrderFirstActionDateTimeUtc" RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_date,
             MAX("OrderFirstActionDateTimeUtc") OVER() AS max_date
             FROM base),
for_rfm AS (SELECT rec.usr,
                   last_date,
                   max_date - last_date AS recency,
                   fm_base.monetary AS monetary,
                   fm_base.frequency AS frequency
            FROM rec
            JOIN fm_base ON rec.usr = fm_base.usr),
r AS (SELECT PERCENTILE_DISC(0.30) WITHIN GROUP (ORDER BY recency) AS r_fs,
             PERCENTILE_DISC(0.70) WITHIN GROUP (ORDER BY recency) AS r_sn
       FROM for_rfm),
rfm AS (SELECT usr,
               last_date,
               recency,
               CASE WHEN recency <= (SELECT r_fs FROM r) THEN '1'
                    WHEN recency <= (select r_sn FROM r) THEN '2'
                    ELSE '3' 
                    END AS R,
               frequency,
               CASE WHEN frequency <= (SELECT fr_fs FROM f) THEN '3'
                    WHEN frequency <= (select fr_sn FROM f) THEN '2'
                    ELSE '1' 
                    END AS F,
               monetary,
               CASE WHEN monetary <= (SELECT mn_fs FROM m) THEN '3'
                    WHEN monetary <= (SELECT mn_sn FROM m) THEN '2'
                    ELSE '1' 
                    END AS M
        FROM for_rfm)
SELECT usr,
       EXTRACT(DAYS FROM recency)::int as recency,
       frequency,
       monetary,
       CONCAT(R,F,M) AS RFM
FROM rfm"""


rfm_segment = sql_query(rfm)

In [ ]:
rfm_segment

## Retention

In [ ]:
# n-month retention
retention = """WITH sessions AS (SELECT "CustomerActionCustomerIdsMindboxId" AS usr,
                                    MIN(date_trunc('month',"CustomerActionDateTimeUtc"::date)) OVER (PARTITION BY "CustomerActionCustomerIdsMindboxId") AS cohort,
                                    date_trunc('month',"CustomerActionDateTimeUtc"::date) AS mth                         
                             FROM actions),
                cohorts AS (SELECT cohort::date as cohort,
                                   mth,
                                   COUNT(DISTINCT usr) as cnt
                            FROM sessions
                            GROUP BY cohort, mth
                            ORDER BY cohort, mth)
SELECT cohort,
       ROW_NUMBER() OVER(PARTITION BY cohort ORDER BY mth) - 1 AS mth,
       ROUND(cnt::numeric / MAX(cnt) OVER(PARTITION BY cohort)::numeric, 4) *100 AS n_month_retention
FROM cohorts
"""

n_mth_retention = sql_query(retention)

In [ ]:
n_mth_retention

## Churn rate

In [ ]:
churn = """WITH profiles AS (SELECT "CustomerActionCustomerIdsMindboxId" AS usr,
                         MIN(DATE_TRUNC('month', "CustomerActionDateTimeUtc"))::date AS cohort
                  FROM actions
                  GROUP BY "CustomerActionCustomerIdsMindboxId"),
     cohorts AS (SELECT cohort,
                        DATE_TRUNC('month', a."CustomerActionDateTimeUtc")::date AS event_mth,
                        COUNT(DISTINCT a."CustomerActionCustomerIdsMindboxId") AS cnt
                 FROM profiles AS p
                 JOIN actions AS a ON p.usr = a."CustomerActionCustomerIdsMindboxId"
                 GROUP BY cohort, event_mth)
SELECT *,
       LAG(cnt) OVER(PARTITION BY cohort ORDER BY event_mth),
       COALESCE(ROUND((1 - (cnt::NUMERIC / LAG(cnt) OVER(PARTITION BY cohort ORDER BY event_mth)))*100, 2), 0) AS churn
FROM cohorts
"""
churn_rate = sql_query(churn)

In [ ]:
churn_rate

## LTV

In [ ]:
ltv = """WITH profiles AS (SELECT "CustomerActionCustomerIdsMindboxId" AS usr,
                                  MIN(DATE_TRUNC('month', "CustomerActionDateTimeUtc"))::date AS cohort
                           FROM actions
                           GROUP BY "CustomerActionCustomerIdsMindboxId"),
              cohorts AS (SELECT p.cohort,
                                 DATE_TRUNC('month', o."OrderFirstActionDateTimeUtc")::date AS event_mth,
                                 COUNT(p.usr) OVER (PARTITION BY p.cohort) AS cnt,
                                 SUM(o."OrderLinePriceOfLine") OVER (PARTITION BY p.cohort ORDER BY (DATE_TRUNC('month', o."OrderFirstActionDateTimeUtc")::date)) AS summa
                          FROM profiles AS p
                          JOIN orders AS o ON p.usr = o."OrderCustomerIdsMindboxId" 
                                           AND o."OrderLineStatusIdsExternalId" LIKE 'Paid'),
              ltv_tbl AS (SELECT cohort,
                                 event_mth,
                                 ROUND(MAX(summa) / MAX(cnt), 2) AS ltv
                          FROM cohorts
                          GROUP BY cohort, event_mth)
SELECT cohort,
       ROW_NUMBER() OVER (PARTITION BY cohort ORDER BY event_mth) - 1 AS mth,
       ltv
FROM ltv_tbl
"""
ltv = sql_query(ltv)

In [ ]:
ltv

## Average order value (AOV)

In [ ]:
aov = """WITH profiles AS (SELECT "CustomerActionCustomerIdsMindboxId" AS usr,
                                  MIN(DATE_TRUNC('month', "CustomerActionDateTimeUtc"))::date AS cohort
                           FROM actions
                           GROUP BY "CustomerActionCustomerIdsMindboxId"), 
              cohorts AS (SELECT p.cohort,
                                 DATE_TRUNC('month', o."OrderFirstActionDateTimeUtc")::date AS event_mth,
                                 COUNT(o."OrderIdsMindboxId") AS cnt,
                                 SUM(o."OrderLinePriceOfLine") AS summa
                          FROM profiles p
                          LEFT JOIN orders o ON p.usr = o."OrderCustomerIdsMindboxId" 
                                             AND o."OrderLineStatusIdsExternalId" like 'Paid'::text
                          GROUP BY p.cohort, (date_trunc('month', o."OrderFirstActionDateTimeUtc")::date))
 SELECT cohort,
        ROW_NUMBER() OVER (PARTITION BY cohort ORDER BY event_mth) - 1 AS mth,
        ROUND(summa /cnt, 2) AS aov
        FROM cohorts"""

aov = sql_query(aov)

In [ ]:
aov

## MAU, WAU, DAU, sticky factor

In [ ]:
mau = """SELECT date_trunc('month',"CustomerActionDateTimeUtc")::date AS mth,
                COUNT(DISTINCT "CustomerActionCustomerIdsMindboxId") AS cnt
         FROM actions
        GROUP BY date_trunc('month',"CustomerActionDateTimeUtc")::date
        ORDER BY mth"""
wau = """SELECT EXTRACT(YEAR FROM "CustomerActionDateTimeUtc") AS yr,
                EXTRACT(WEEK FROM "CustomerActionDateTimeUtc") AS wk,
                COUNT(DISTINCT "CustomerActionCustomerIdsMindboxId") AS cnt
         FROM actions
         GROUP BY EXTRACT(YEAR FROM "CustomerActionDateTimeUtc"), EXTRACT(WEEK FROM "CustomerActionDateTimeUtc")
         ORDER BY yr, wk"""
dau = """SELECT "CustomerActionDateTimeUtc"::date AS dt,
                 COUNT(DISTINCT "CustomerActionCustomerIdsMindboxId") AS cnt
         FROM actions
         GROUP BY "CustomerActionDateTimeUtc"::date
         ORDER BY dt"""
sticky_factor = """WITH mau AS (SELECT date_trunc('month',"CustomerActionDateTimeUtc")::date AS mth,
                                       COUNT(DISTINCT "CustomerActionCustomerIdsMindboxId") AS cnt
                                FROM actions
                                GROUP BY date_trunc('month',"CustomerActionDateTimeUtc")::date),
                        dau AS (SELECT "CustomerActionDateTimeUtc"::date AS dt,
                                       COUNT(DISTINCT "CustomerActionCustomerIdsMindboxId") AS cnt
                                FROM actions
                                GROUP BY "CustomerActionDateTimeUtc"::date
                                ORDER BY dt),
                        avgdau AS (SELECT date_trunc('month', dt) AS mth,
                                          avg(cnt) AS cnt
                                   FROM dau
                                   GROUP BY date_trunc('month', dt))
                  SELECT m.mth,
                         round(a.cnt/m.cnt * 100, 2) AS sticky_factor
                  FROM mau AS m
                  JOIN avgdau AS a ON m.mth = a.mth"""

In [ ]:
mau = sql_query(mau)
wau = sql_query(wau)
dau = sql_query(dau)
sticky_factor = sql_query(sticky_factor)

In [ ]:
mau

In [ ]:
wau

In [ ]:
dau

In [ ]:
sticky_factor